# Mosquito arms

In [1]:
# Don't change this cell; just run it.
import numpy as np  # The array library.
# Make a random number generator.
rng = np.random.default_rng()

import pandas as pd
# Safe setting for Pandas.
pd.set_option('mode.copy_on_write', True)

import matplotlib.pyplot as plt
%matplotlib inline

# The OKpy testing system.
from client.api.notebook import Notebook
ok = Notebook('mosquito_arms.ok')

We are sure you remember the [mosquito / beer study](https://lisds.github.io/textbook/permutation/permutation_idea) with affection.

As before, see [this
page](https://github.com/matthew-brett/datasets/tree/master/mosquito_beer) for
more details on the dataset.

The paper describing the data is [Beer Consumption Increases Human
Attractiveness to Malaria
Mosquitoes](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0009546).

Read in the data:

In [2]:
mosquitoes = pd.read_csv('mosquito_beer.csv')
mosquitoes.head()

## The experiment

The authors studied whether people who had drunk beer were more attractive to
mosquitoes.

We didn't go into details in the textbook page, but in fact the experimental
setup was a little more complicated than we described.

Here is the graphic from the paper:

![](experimental_setup.png)

For each trial, there were two tents.

* One tent was empty (the *control* tent).
* The other tent contained a person (the *volunteer* tent).
* A tube led from each tent to a corresponding *trap* box. Thus, there was a
  *control trap* box and a *volunteer trap* box.
* A tube from each trap box fed into an arm of a Y connector.
* The remaining, third arm of the Y led to a *downwind box* containing 50
  mosquitoes.
* At the beginning of the trial, the experimenters opened the *downwind box*
  of mosquitoes, so the mosquitoes could fly out into the Y connector, and
  thence, into either of the *trap* boxes.
* The number of mosquitoes who flew into the *control trap* box gives the
  values for the `no_odour` column.
* The number of mosquitoes who flew into the *volunteer trap* box gives the
  values for the `volunt_odour` column.
* The total number of mosquitoes who flew into either the trap box gives
  the values for the `activated` column.

Each volunteer had two trials, one *before* they drank their allocated drink,
and one *after* they drank their allocated drink.  The `test` columns records
the trial type of each row.  The study allocated 25 volunteers to drink beer,
and 18 volunteers to drink water.  The `group` column contains the allocation
of the corresponding volunteer on each trial.

## A check

From what we have said above, you might assume that the mosquitoes who left
their box (`activated` number) would always equal the number of who flew to
the `no_odour` arm plus the number who flew to the `volunt_odour` arm.

Check this by adding the values in `no_odour` to those in `volunt_odour`, then
comparing them for equality, to the `activated` numbers.  Finally count the
number of values where you got True for this comparison.  It should be the
same as the number of rows in the data frame - if this relationship holds for
all rows.

In [3]:
both = mosquitoes['no_odour'] + mosquitoes['volunt_odour']
same = both == mosquitoes['activated']
n_same = np.count_nonzero(same)
# Show the result
n_same

In [4]:
_ = ok.grade('q_n_same')

## Another test for beer

A comparison of interest to the authors was the difference between the number
of mosquitoes that flew towards the volunteer and the number that flew towards
the empty tent.

In the next cell, first select the rows corresponding to the trials for
volunteers after they had had their allocated drink.

Next make a new column in the resulting new data frame. Call the column
`volunt_diff`.  The column should have the result of the subtraction of
the numbers in `no_odour` from those in `volunt_odour`.

In [5]:
afters = mosquitoes[mosquitoes['test'] == 'after']
afters['volunt_diff'] = afters['volunt_odour'] - afters['no_odour']
# Show the first five rows.
afters.head()

In [6]:
_ = ok.grade('q_afters')

Make an array from the `volunt_diff` values for the rows corresponding to the
`beer` drinkers and another for the rows corresponding to the water drinkers.
Call these arrays `after_beer_vd` and `after_water_vd` respectively.

In [7]:
after_beer = afters[afters['group'] == 'beer']
after_beer_vd = np.array(after_beer['volunt_diff'])
after_water = afters[afters['group'] == 'water']
after_water_vd = np.array(after_water['volunt_diff'])
# Show the results
print(after_beer_vd)
print(after_water_vd)

In [8]:
_ = ok.grade('q_after_arrs')

## More permutation

Consider the mean difference between the differences:

In [9]:
beer_mean = np.mean(after_beer_vd)
water_mean = np.mean(after_water_vd)
beer_water_diff = beer_mean - water_mean
beer_water_diff

The number of beer values:

In [10]:
n_beer = len(after_beer_vd)
n_beer

The values, pooled into one array:

In [11]:
# The values, pooled.
pooled = np.concatenate([after_beer_vd, after_water_vd])
pooled

Your job is to do a *permutation* test, to see whether this observed mean
difference is plausible in an ideal (null) world, where there is no real
difference between the groups, and any observed difference is just due to
random sampling.

We simulate samples from such an ideal world by shuffling the 16 values
randomly, allocating 25 shuffled values to a fake Beer group, and the rest to 
fake Water group, and calculating the mean difference for these fake groups.
We do this many times to build up the *sampling distribution* of these fake
differences.

To do this job, you may want to remind yourself of the [permutation idea](https://lisds.github.io/textbook/permutation/permutation_idea.html) notebook in the textbook.

You may well want to start with a cell that does one trial where you:

* shuffle the pooled values.
* split them into two groups of 25 and 18.
* calculate the difference in means.

In [12]:
#- You may want to simulate a single trial here.
shuffled = rng.permutation(pooled)
fake_diff = np.mean(shuffled[:n_beer]) - np.mean(shuffled[n_beer:])
fake_diff

Then finish up the cell below to build your sampling distribution, storing the values in the array `fake_diffs`.

In [13]:
# Build up the sampling distribution from the ideal (null) world.
n_iters = 10000
fake_diffs = np.zeros(n_iters)
for i in np.arange(n_iters):
    shuffled = rng.permutation(pooled)
    fake_diffs[i] = np.mean(shuffled[:n_beer]) - np.mean(shuffled[n_beer:])
# Show the first 10 values.
fake_diffs[:10]

In [14]:
_ = ok.grade('q_fake_diffs')

You might also like to review the histogram of these values, to compare by eye to the value in the real world, `beer_water_diff`.

In [15]:
#- Do a histogram of the sampling distribution here.
plt.hist(fake_diffs)

Calculate the proportion of the sampling distribution values that are greater than or equal to the observed difference in means.

In [16]:
prop_ge = np.count_nonzero(fake_diffs >= beer_water_diff) / n_iters
# Show the proportion.
prop_ge

In [17]:
_ = ok.grade('q_prop_ge')

## Done.

Congratulations, you're done with the assignment!  Be sure to:

- **run all the tests** (the next cell has a shortcut for that).
- **Save and Checkpoint** from the `File` menu.

In [18]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]